In [ ]:
# import our packages
import pandas as pd
import requests
import os  
import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import boxscoretraditionalv2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Get All Player Data 

In [ ]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [ ]:
if os.path.exists("./NBA_Stats/player_stats.csv"):
  os.remove("./NBA_Stats/player_stats.csv")
else:
  print("The file does not exist") 

In [ ]:
player_dictionary = players.get_players()

for player in player_dictionary:
    try:    
        if player['is_active'] == True:
            print(player['last_name'])
            player_log = playergamelog.PlayerGameLog(
            season = '2022',
            player_id = player['id'],
            season_type_all_star = ['Regular Season'])
            player_log.get_data_frames()[0].to_csv(f'./NBA_Stats/player_stats.csv', mode='a', index=False, header=True)

            time.sleep(.600)
    except:
        continue

In [ ]:
df = pd.read_csv('./NBA_Stats/player_stats.csv')
grouped = df.groupby("Player_ID")
grouped.head()

Add Game Count Coloumn

In [ ]:
df["Game Count"] = grouped["Game_ID"].cumcount() + 1


Find Duplicate Headers & Put into player_stats.csv

In [ ]:
# find the indices of the rows that contain the headers
header_rows = [i for i, row in df.iterrows() if row.astype(str).str.contains('PTS|REB|AST').any()]

# drop the rows that contain the headers
df.drop(header_rows, inplace=True)
df.to_csv('./NBA_Stats/player_stats.csv', index=False)

Build Out AVG Stats CSV

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the csv file into a pandas DataFrame
df = pd.read_csv("./NBA_Stats/player_stats.csv")

# Group the data by player_id
grouped_df = df.groupby("Player_ID")

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=["Player_ID"])

# Loop through each player's data
for Player_ID, player_data in grouped_df:
    #check to make sure there is enough data
    if len(player_data) >= 20:
        # Calculate the average of the last 7 games for each player
        totalGamesPlayed = len(player_data)
        totalAvgPTS = player_data['PTS'].mean()
        totalAvgAST = player_data['AST'].mean()
        totalAvgREB = player_data['REB'].mean()
        sevenGameAvg_MIN = player_data.sort_values(by="Game_ID", ascending=False).head(7)["MIN"].mean()
        tenGameAvg_MIN = player_data.sort_values(by="Game_ID", ascending=False).head(10)["MIN"].mean()
        fifteenGameAvg_MIN = player_data.sort_values(by="Game_ID", ascending=False).head(15)["MIN"].mean()
        twentyGameAvg_MIN = player_data.sort_values(by="Game_ID", ascending=False).head(20)["MIN"].mean()        
        sevenGameAvg_PTS = player_data.sort_values(by="Game_ID", ascending=False).head(7)["PTS"].mean()
        sevenGameAvg_REB = player_data.sort_values(by="Game_ID", ascending=False).head(7)["REB"].mean()
        sevenGameAvg_AST = player_data.sort_values(by="Game_ID", ascending=False).head(7)["AST"].mean()
        tenGameAvg_PTS = player_data.sort_values(by="Game_ID", ascending=False).head(10)["PTS"].mean()
        tenGameAvg_REB = player_data.sort_values(by="Game_ID", ascending=False).head(10)["REB"].mean()
        tenGameAvg_AST = player_data.sort_values(by="Game_ID", ascending=False).head(10)["AST"].mean()
        fifteenGameAvg_PTS = player_data.sort_values(by="Game_ID", ascending=False).head(15)["PTS"].mean()
        fifteenGameAvg_REB = player_data.sort_values(by="Game_ID", ascending=False).head(15)["REB"].mean()
        fifteenGameAvg_AST = player_data.sort_values(by="Game_ID", ascending=False).head(15)["AST"].mean()
        twentyGameAvg_PTS = player_data.sort_values(by="Game_ID", ascending=False).head(20)["PTS"].mean()
        twentyGameAvg_REB = player_data.sort_values(by="Game_ID", ascending=False).head(20)["REB"].mean()
        twentyGameAvg_AST = player_data.sort_values(by="Game_ID", ascending=False).head(20)["AST"].mean()
        
        # Count the number of games where the player did not reach their average
        below_7avgPTS_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(7).query("PTS < @totalAvgPTS"))
        below_10avgPTS_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(10).query("PTS < @totalAvgPTS"))
        below_15avgPTS_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(15).query("PTS < @totalAvgPTS"))
        below_20avgPTS_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(20).query("PTS < @totalAvgPTS"))
        below_7avgAST_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(7).query("AST < @totalAvgAST"))
        below_10avgAST_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(10).query("AST < @totalAvgAST"))
        below_15avgAST_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(15).query("AST < @totalAvgAST"))
        below_20avgAST_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(20).query("AST < @totalAvgAST"))
        below_7avgREB_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(7).query("REB < @totalAvgREB"))
        below_10avgREB_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(10).query("REB < @totalAvgREB"))
        below_15avgREB_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(15).query("REB < @totalAvgREB"))
        below_20avgREB_count = len(player_data.sort_values(by="Game_ID", ascending=False).head(20).query("REB < @totalAvgREB"))

        # Calculate the % a player hit thier avg in the last 7,10,15,20 games
        hitAvg7Percent = (below_7avgPTS_count/7)
        hitAvg10Percent = (below_10avgPTS_count/10)
        hitAvg15Percent = (below_15avgPTS_count/15)
        hitAvg20Percent = (below_20avgPTS_count/20)
        hitAvg7REBPercent = (below_7avgREB_count/7)
        hitAvg10REBPercent = (below_10avgREB_count/10)
        hitAvg15REBPercent = (below_15avgREB_count/15)
        hitAvg20REBPercent = (below_20avgREB_count/20)
        hitAvg7ASTPercent = (below_7avgAST_count/7)
        hitAvg10ASTPercent = (below_10avgAST_count/10)
        hitAvg15ASTPercent = (below_15avgAST_count/15)
        hitAvg20ASTPercent = (below_20avgAST_count/20)


        # Store the results for each player in the results DataFrame
        results_df = results_df.append({
            "Player_ID": Player_ID,
            "Numb_Games_Played": totalGamesPlayed,
            "AVG_Points": totalAvgPTS,
            "Below_AvgPTS_Past_7": below_7avgPTS_count,
            "HitAvgPercent_Past7": hitAvg7Percent,
            "Below_AvgPTS_Past_10": below_10avgPTS_count,
            "HitAvgPercent_Past10": hitAvg10Percent,
            "Below_AvgPTS_Past_15": below_15avgPTS_count,
            "HitAvgPercent_Past15": hitAvg15Percent,
            "Below_AvgPTS_Past_20": below_20avgPTS_count,
            "HitAvgPercent_Past20": hitAvg20Percent,
            "7_Game_avg_PTS": sevenGameAvg_PTS, 
            "10_Game_avg_PTS": tenGameAvg_PTS,
            "15_Game_avg_PTS": fifteenGameAvg_PTS,
            "20_Game_avg_PTS": twentyGameAvg_PTS,
            "Below_AvgREB_Past_7": below_7avgREB_count,
            "HitAvgREB7_%": hitAvg7REBPercent,
            "HitAvgREB10_%": hitAvg10REBPercent,
            "HitAvgREB15_%": hitAvg15REBPercent,
            "HitAvgREB20_%": hitAvg20REBPercent,
            "HitAvgAST7_%": hitAvg7ASTPercent,
            "HitAvgAST10_%": hitAvg10ASTPercent,
            "HitAvgAST15_%": hitAvg15ASTPercent,
            "HitAvgAST20_%": hitAvg20ASTPercent,
            "Below_AvgREB_Past_10": below_10avgREB_count,
            "Below_AvgREB_Past_15": below_15avgREB_count,
            "Below_AvgREB_Past_15": below_20avgREB_count,
            "7_Game_avg_REB": sevenGameAvg_REB, 
            "10_Game_avg_REB": tenGameAvg_REB,
            "15_Game_avg_REB": fifteenGameAvg_REB,
            "20_Game_avg_REB": twentyGameAvg_REB,
            "Below_AvgAST_Past_7": below_7avgAST_count,
            "Below_AvgAST_Past_10": below_10avgAST_count,
            "Below_AvgAST_Past_15": below_15avgAST_count,
            "Below_AvgAST_Past_15": below_20avgAST_count,
            "7_Game_avg_AST": sevenGameAvg_AST,
            "10_Game_avg_AST": tenGameAvg_AST,
            "15_Game_avg_AST": fifteenGameAvg_AST,
            "20_Game_avg_AST": twentyGameAvg_AST,
            "7_Game_avg_MIN": sevenGameAvg_MIN,
            "10_Game_avg_MIN": tenGameAvg_MIN,
            "15_Game_avg_MIN": fifteenGameAvg_MIN,
            "20_Game_avg_MIN": twentyGameAvg_MIN
            }, ignore_index=True)

# Save the results to a new csv file
results_df.to_csv("./NBA_Stats/player_avg_stats.csv", index=False)
